### web scraping (só pelo o id)

In [22]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

In [ ]:
import time
def url_get(i):
    url_list= [ ]
    for id in range(243,(243+i)):
        url = "https://www.ncbi.nlm.nih.gov/nuccore/NG_009{}".format( id )
        url_list.append(url)
    return url_list
url_get(4)

In [ ]:
import requests

content = []
for url in url_get(4):
    r = requests.get(url)
    content.append(r.content)
print(content)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id = line.attrs['content']
    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

    #r2 = requests.get( url )
    
    #r3= str(r2.content)
    #print (r3, '\n')

In [ ]:
#import re
#existe = re.findall(r"ORGANISM\s+.*?(?=bp)", r3, re.DOTALL)
#if existe:
    #for definition in existe:
        #m = re.match( r"ORGANISM\s+(.+)", definition, re.DOTALL )
        #print( re.sub(r'\s+', ' ', m.group(1) ) )

### web scraping (por condição)

In [44]:
import time
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

escolha o que quer pesquisar: yeast


['https://www.ncbi.nlm.nih.gov/gene/?term=yeast']

In [45]:
import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

In [46]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    

In [47]:
import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)
#existe
#if existe:
    #for definition in existe:
        #m = re.match( r"ID:\s+(.+)", definition, re.DOTALL )
        #print( re.sub(r'\s+', ' ', m.group(1) ) )


In [48]:
c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')
#IDS

In [67]:
n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
print(n_genes)

escolha o nº de genes que quer obter (máximo 20): 20
['424122', '29789', '26245415', '20918', '24563533', '39874648', '4839978', '42049400', '42048491', '36516274', '35606568', '35598500', '24564880', '24564814', '13446414', '5477914', '65082716', '109860985', '8047674', '4292']


### Buscar Links dos genes:

In [59]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in n_genes:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(n_genes)

['https://www.ncbi.nlm.nih.gov/nuccore/424122',
 'https://www.ncbi.nlm.nih.gov/nuccore/29789']

In [ ]:
import requests

content_id = []
for url in url_get_id(len(IDS)):
    r = requests.get(url)
    content_id.append(r.content)
#print(content_id)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines_id = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id_id = ""
    url = ""
    for line in lines_id:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id_id = line.attrs['content']
    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id_id)

    r2 = requests.get( url )
    r3= str(r2.content)
    print (r3, '\n')

In [ ]:
IDS

### Buscar ids:

In [60]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    print(info.id)

U02075
X07874.2


### Buscar quantidade de cds e a sua localização:

In [52]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
pos=[]
for info in records:
    print(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            featcds.append(i)
            cds= i
            a= str(cds)
            print (len(a))
            position=i
            pos.append(info.features[position].location)
            print (pos)

U02075
X07874.2
AY158036.1
BW173609.1


### Buscar quantidade de cds e a sua localização:

In [61]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

U02075
GGATCCTGCCAGTAGTCATATGCTTGTCTCAAAGACTAAGCCATGCATGT ... GAACCTGCAG

X07874.2
TATTGAGGTTTTGTTGTTGCAGATGAGGAGCTGGAGACAAGAGCCCACAG ... GCTCATTGTG



### Buscar gene description:

In [62]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    print(info.id)
    print(info.description)
    description.append(info.description)

U02075
Chlorarachnion sp. 1 CCMP 1408 clones pF58 and pF59 18S rRNA small subunit
X07874.2
Human CD2 gene exon 5 and 3'-flanking region


### Buscar artigos:

In [75]:
Ids=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    print("title:", info.get("TI", "-"))
    print('authors: ', info.get("AU", "-"))
    print("source:", info.get("SO", "-"))
    print("affiliation: ", info.get("AD", "-") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
    print()

title: Postpartum septicemia due to group G streptococci.
authors:  ['Filker RS', 'Monif GR']
source: Obstet Gynecol. 1979 Mar;53(3 Suppl):28S-30S.
affiliation:  -

title: PMS photo-inhibition in Rhodospirillum rubrum membranes in the presence of permeant entities affecting either the deltapsi or the deltapH components of the protonmotive force.
authors:  ['Kerber NL', 'Pucheu NL', 'Garcia AF']
source: FEBS Lett. 1978 Oct 15;94(2):265-8. doi: 10.1016/0014-5793(78)80952-1.
affiliation:  -

title: An examination of the factors fueling migration amongst Community Service practitioners.
authors:  ['Reardon C', 'George G']
source: Afr J Prim Health Care Fam Med. 2014 Nov 7;6(1):E1-9. doi: 10.4102/phcfm.v6i1.625.
affiliation:  ['Health Economics and HIV and AIDS Research Division (HEARD), University of KwaZulu-Natal. georgeg@ukzn.ac.za.']

title: Clobazam, a 1,5-benzodiazepine, and car-driving ability.
authors:  ['Hindmarch I', 'Hanks GW', 'Hewett AJ']
source: Br J Clin Pharmacol. 1977 Oct;4